<a href="https://colab.research.google.com/github/dkanh6/Computational_Neuroscience/blob/Custom/HCO_Single_Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Single Cell Spiking Model**

This script will model a single compartment neuron that will have classic AP spiking activity.


## **Notebook Setup**
---

### **Install Neuron**
first install NEURON package, import .h submodule, and matplotlib + numpy


In [13]:
!pip install neuron
from neuron import h
import matplotlib.pyplot as plt
import numpy as np
h.load_file('stdrun.hoc')

1.0

## **Model Setup**
This will be a simple single compartment cell model


### **Initalize the Model**


In [32]:
### Creating the Cell Body ###
soma = h.Section(name='soma')
soma.nseg = 4 # determines how many indivdual segments to divide a given Section into

print("*******Default Soma Values*******") # These values are not realistic for a human neuron as these values are derived from the Hodgkin Huxley Squid Giant Neuron experimental alue
print("Length:             {}  um".format(str(soma.L)))
print("Diamter:            {}  um".format(str(soma.diam)))
print("Capacitance:        {}  uF/cm^2".format(str(soma.cm)))
print("Axial Resistance:   {}  ohm-cm\n".format(str(soma.Ra)))


soma.L = 30 #um
soma.diam = 30 #um
print("*******Adjusted Soma Values*******") # These
print("Length:             {}  um".format(str(soma.L)))
print("Diamter:            {}  um".format(str(soma.diam)))
print("Capacitance:        {}  uF/cm^2".format(str(soma.cm)))
print("Axial Resistance:   {}  ohm-cm\n".format(str(soma.Ra)))



*******Default Soma Values*******
Length:             100.0  um
Diamter:            500.0  um
Capacitance:        1.0  uF/cm^2
Axial Resistance:   35.4  ohm-cm

*******Adjusted Soma Values*******
Length:             30.0  um
Diamter:            30.0  um
Capacitance:        1.0  uF/cm^2
Axial Resistance:   35.4  ohm-cm

